In [14]:
import os
import json
import shelve

import warnings
warnings.filterwarnings("ignore")

import torch
import numpy as np
from models.rgcn import RGCN

from prolog.prolog_generation import PrologData

from data.ag.action_genome import AG, AGViewer

%load_ext autoreload

pygame 2.6.1 (SDL 2.30.7, Python 3.9.20)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
%autoreload
# Load dataset
root = '/data/Datasets/ag/'
ag = AG(root, no_img=True)

In [3]:
# Load model
#model = RGCN(ag.num_nodes, ag.num_rels, ag.num_classes, ag.num_bases, ag.num_hidden, ag.num_layers, ag.dropout, ag.use_self_loop)
model = None

In [4]:
%autoreload
# Load prolog data
pd = PrologData('prolog', 'ag', ag, ag.object_classes, ag.relationship_classes, ag.verb_classes, 
                model=None, subset_file='data/ag/subset_shelve')

pd.write_bk()
pd.init_general_bias()

In [5]:
%autoreload
#need the verb count
with shelve.open('data/ag/subset_shelve') as subset_dict:
    agview = AGViewer(ag, subset_dict=subset_dict)
    total_valid, _, verb_freq, _ = agview.analyze_vocab_frequencies()


In [6]:
def exp_curve(b,x):
    return 1-np.exp(-b*x)

for verb_idx, verb_name in enumerate(ag.verb_classes):

    freq = verb_freq[verb_idx]
    ratio = freq/total_valid
    
    #keeps negatives according to the frequency of the verb
    pd.write_verb(verb_name, keep_prob=exp_curve(4, ratio)) 

In [17]:
def parse_logs(folder):
    rules = {}
    for file in os.listdir(folder):
        rule = []
        with open(folder + '/' + file, 'r') as f:
            lines = f.readlines()
            if lines[-1].strip('\n') != "******************************":
                print(file, 'no solution')
                rules[file] = None
                continue
            for line in reversed(lines[:-1]):
                line = line.strip('\n')
                if line[-1] != '.':
                    break
                rule.append(line)
        rules[file] = rule
    return rules

rules = parse_logs('outputs/ag/popper_logs')

with open('outputs/ag/rules_learned.json', 'w+') as f:
    json.dump(rules, f)

with open('outputs/ag/rules_learned.pl', 'w+') as f: 
    for verb, rule in rules.items():
        f.write(f'%%{verb}\n')
        if rule is None:
            f.write('%%No solution\n\n')
            continue
        for line in rule:
            f.write(line + '\n')
        f.write('\n')

pour no solution
drink no solution
make no solution
laugh no solution
put no solution
run no solution
talk no solution
photograph no solution


In [13]:
rules

{'sneeze': ['sneeze_target(A):- touching(A,B),mirror(B).'],
 'undress': ['undress_target(A):- person(B),wearing(B,C),wearing(A,C).'],
 'throw': ['throw_target(A):- standing_on(A,B),floor(B).',
  'throw_target(A):- clothes(B),holding(A,B).',
  'throw_target(A):- pillow(B),holding(A,B).'],
 'smile': ['smile_target(A):- not_contacting(A,B),picture(B).',
  'smile_target(A):- holding(A,B),mirror(B).',
  'smile_target(A):- touching(A,B),broom(B).',
  'smile_target(A):- blanket(B),other_relationship(A,B).'],
 'work': ['work_target(A):- table(B),in_front_of(B,A).'],
 'close': ['close_target(A):- in_front_of(B,A),refrigerator(B).',
  'close_target(A):- in_front_of(B,A),door(B).',
  'close_target(A):- on_the_side_of(B,A),refrigerator(B).',
  'close_target(A):- on_the_side_of(B,A),door(B).'],
 'take': ['take_target(A):- on_the_side_of(B,A),holding(A,B),food(B).',
  'take_target(A):- in_front_of(B,A),clothes(B),touching(A,B).',
  'take_target(A):- blanket(B),in_front_of(B,A),not_contacting(A,B).',

In [9]:
arr = [1,2,3,4,5,6]
arr[-1]

6